# Base parser (Part 2 - Upload annotations)

## Imports (Customizable)

In [ ]:
import jdc

from dtlpylidar.parser_base import extrinsic_calibrations
from dtlpylidar.parser_base import images_and_pcds, camera_calibrations, lidar_frame, lidar_scene
import dtlpylidar.utilities.transformations as transformations
import dtlpy as dl
import pandas as pd
import numpy as np
import os
import json
import logging
import shutil
import pathlib

logger = logging.Logger(name="lidar_base_parser")

## Get Dataloop dataset and remote path (On the Dataloop dataset)

Get the target Dataloop dataset to build the lidar scene for, with the path to the folder where the lidar scene files are located in the target Dataloop dataset.
`Note:` This time we will need the `frames.json` item to upload the annotations to it.

In [ ]:
dataset_id = ""  # TODO: fill with dataset id
dataset = dl.datasets.get(dataset_id=dataset_id)
remote_path: str = "/"
item_id = ""  # TODO: fill with frames.json item id
frames_item = dl.items.get(item_id=item_id)

## Download data (Customizable)

`Note:` This time we will use this method to download the annotation files.

In [ ]:
class LidarBaseParser(dl.BaseServiceRunner):
    @staticmethod
    def download_data(dataset: dl.Dataset, remote_path: str, download_path: str) -> tuple:
        """
        Download the required data for the parser
        :param dataset: Input dataset
        :param remote_path: Path to the remote folder where the lidar data is uploaded
        :param download_path: Path to the downloaded data
        :return: (items_path, json_path) Paths to the downloaded items and annotations JSON files directories
        """
        # Download items dataloop annotation JSONs
        # (PCD and Image annotation JSONs contains the Dataloop platform references (Like: ID) to the remote files)
        filters = dl.Filters(field="metadata.system.mimetype", values="*pcd*", method=dl.FiltersMethod.OR)
        filters.add(field="metadata.system.mimetype", values="*image*", method=dl.FiltersMethod.OR)
        dataset.download_annotations(local_path=download_path, filters=filters)

        # Download required binaries (Calibrations Data)
        # Pandaset Calibration Data is saved in JSON files (Like: poses.json, intrinsics.json, timestamps.json)
        filters = dl.Filters(field="metadata.system.mimetype", values="*json*")
        dataset.items.download(local_path=download_path, filters=filters)

        # Download required binaries (Annotations Data)
        # Pandaset Annotations Data is saved in CSV files (Like: 01.csv in cuboids folder)
        filters = dl.Filters(field="metadata.system.mimetype", values="*csv*")
        dataset.items.download(local_path=download_path, filters=filters)

        items_path = os.path.join(download_path, "items", remote_path)
        json_path = os.path.join(download_path, "json", remote_path)
        return items_path, json_path

### Run to download annotation files

In [ ]:
parser = LidarBaseParser()

if remote_path.startswith("/"):
    remote_path = remote_path[1:]

if remote_path.endswith("/"):
    remote_path = remote_path[:-1]

download_path = os.path.join(os.getcwd(), dataset.name)

items_path, json_path = parser.download_data(
    dataset=dataset,
    remote_path=remote_path,
    download_path=download_path,
)
print(items_path + "\n" + json_path)

## Parse annotations (Customizable)

A method to parse the annotations data from all the downloaded files. \
`Notices:`

1. It is possible to modify the function to upload 3D annotations to the 3D point cloud files and 2D annotations to the images.

2. Annotations uploaded to the separated point cloud and images files will not be visible on the frames.json file. 

In [ ]:
%%add_to LidarBaseParser
@staticmethod
def parse_annotations(frames_item: dl.Item, items_path: str, json_path: str):
    """
    Parse the annotations data to build and upload the annotations to the frames.json item
    :param items_path: Paths to the downloaded items directory
    :param json_path: Paths to the downloaded annotations JSON files directory
    :return: None
    """
    # annotations_json_path = os.path.join(json_path, "annotations")
    annotations_items_path = os.path.join(items_path, "annotations")

    builder = frames_item.annotations.builder()
    frames_json_data = json.loads(s=frames_item.download(save_locally=False).getvalue())

    next_object_id = 0
    uid_to_object_id_map = dict()
    labels = set()

    # Parse the cuboid annotations and add them to the annotations builder
    cuboids_items_path = os.path.join(annotations_items_path, "cuboids")
    cuboids_csvs = pathlib.Path(cuboids_items_path).rglob('*.csv')
    cuboids_csvs = sorted(cuboids_csvs, key=lambda x: int(x.stem))

    for csv_frame_idx, cuboids_csv in enumerate(cuboids_csvs):
        # Getting the lidar scene frame Translation and Rotation
        frame_pcd_translation = frames_json_data["frames"][csv_frame_idx]["translation"]
        frame_pcd_translation = np.array(
            [frame_pcd_translation["x"], frame_pcd_translation["y"], frame_pcd_translation["z"]]
        )
        frame_pcd_rotation = frames_json_data["frames"][csv_frame_idx]["rotation"]
        frame_pcd_rotation = np.array(
            [frame_pcd_rotation["x"], frame_pcd_rotation["y"], frame_pcd_rotation["z"], frame_pcd_rotation["w"]]
        )

        # Opening the current scene frame, cuboid annotations CSV file to get the cuboids annotation data
        cuboids_csv_data = pd.read_csv(filepath_or_buffer=cuboids_csv)
        for _, row_data in cuboids_csv_data.iterrows():
            object_id = uid_to_object_id_map.get(row_data["uuid"], None)
            if object_id is None:
                object_id = next_object_id
                uid_to_object_id_map[row_data["uuid"]] = object_id
                next_object_id += 1

            ann_label = row_data["label"]
            ann_position = np.array([row_data["position.x"], row_data["position.y"], row_data["position.z"]])
            ann_quaternion = transformations.quaternion_from_euler(*[0, 0, row_data["yaw"]])
            ann_scale = np.array([row_data["dimensions.x"], row_data["dimensions.y"], row_data["dimensions.z"]])

            # Calculate the transform matrix of the cuboid annotation relatively to the Scene Frame
            ann_transform_matrix = transformations.calc_cuboid_scene_transform_matrix(
                cuboid_position=ann_position,
                cuboid_quaternion=ann_quaternion,
                cuboid_scale=ann_scale,
                scene_position=frame_pcd_translation,
                scene_quaternion=frame_pcd_rotation
            )

            # Extract the cuboid Translation and Rotation from the transform matrix
            ann_position = transformations.translation_vector_from_transform_matrix(
                transform_matrix=ann_transform_matrix
            )
            ann_rotation_matrix = transformations.rotation_matrix_from_transform_matrix(
                transform_matrix=ann_transform_matrix
            )
            ann_rotation = transformations.euler_from_rotation_matrix(rotation_matrix=ann_rotation_matrix)

            # Add the cuboid annotation to the annotations builder
            annotation_definition = dl.Cube3d(
                label=ann_label,
                position=ann_position,
                scale=ann_scale,
                rotation=ann_rotation
            )
            builder.add(
                annotation_definition=annotation_definition,
                object_id=object_id,
                frame_num=csv_frame_idx
            )
            labels.add(ann_label)

    builder.upload()
    frames_item.dataset.update_labels(label_list=list(labels), upsert=True)

### Test

In [ ]:
parser = LidarBaseParser()

parser.parse_annotations(frames_item=frames_item, items_path=items_path, json_path=json_path)
# frames_item.open_in_web()

## Cleanup

Delete the directory with all the downloaded files.

In [ ]:
shutil.rmtree(path=download_path, ignore_errors=True)